In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd

import sklearn.linear_model as linear_model
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


import typing as tp
from collections import Counter
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler

plt.style.use('bmh')
import statsmodels.api as sm
sns.set_style("darkgrid")
mpl.rcParams['figure.figsize'] = (20,5)

In [2]:
train = pd.read_csv('train.csv' )
print(train.describe())
train.head()
print(f'Train data shape: {train.shape}\n')
print(f'Colums: {train.columns}\n')
print(f'DataFrame information: {train.info()}\n')

train.describe(include=['O']).T
# What we see here is a general description of the individual variables (count, mean, STD, min, max, 25%, 50%, 75%)
# column names, non-null count and data type, # unique, top value and frequency

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ..

,count,unique,top,freq
MSZoning,1460,5,RL,1151
Street,1460,2,Pave,1454
Alley,91,2,Grvl,50
LotShape,1460,4,Reg,925
LandContour,1460,4,Lvl,1311
Utilities,1460,2,AllPub,1459
LotConfig,1460,5,Inside,1052
LandSlope,1460,3,Gtl,1382
Neighborhood,1460,25,NAmes,225
Condition1,1460,9,Norm,1260


In [7]:
train[['Id','Utilities']]


,Id,Utilities
0,1,AllPub
1,2,AllPub
2,3,AllPub
3,4,AllPub
4,5,AllPub
...,...,...
1455,1456,AllPub
1456,1457,AllPub
1457,1458,AllPub
1458,1459,AllPub


In [ ]:
'We can see here that there are various columns with null entries, some of which with mostly null entries (Alley). We can also see how many unique values there are per feature and the most common frequency. From that, we can see that Street has 2 unique values, 6 of the values are Grvl, so we can remove Street as it does not provide enough data to be useful and the same thing for Utilities. As for the null values, we can see the nulls are for when that feature is not there, hence null. For example, PoolQC is null when there is no pool.'

In [ ]:
# It's important to understand how to handle the gaps so as to minimize the model prediction error
columns_count = len(train.columns)
pd.options.display.max_rows = columns_count

missing = train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()
# We can see that the values are correlated (BsmtQual, BsmtCond, BsmtFinType1,  BsmtExposure,  BsmtFinType2 (last 2 have additional null))
# (GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond), MasVnrType, MasVnrArea (could be zero??)
#LotFrontage, Alley, FireplaceQu, PoolQC, Fence, MiscFeature, Electrical not correlated with others but seem to indicate whether it doesn't have these features
# Perhaps, some people simply do not have it, so there are obvious gaps in the data in the footer features \ Therefore, here, in order to avoid data corruption, 
#it is worth using the MAR (Missing At Random) method for processing gaps. Since such data may exist: a set of features is simply not allows you to have a 
#basement, say, the house is not private, but a high-rise building
#Electrical is replaced by the most common values
#There is no alley, most likely because the house is not private

In [ ]:
'It is important to understand how to handle gaps so as to minimize model prediction error. 19 attributes have missing values, 5 over 50% of all data. NA means lack of subject described by the attribute, like missing pool, fence, no garage and basement.'

In [ ]:
train = train.drop(['Utilities'], axis=1) #there's not enough diverse data in Utilities and Street to make meaningful changes
train = train.drop(['Street'], axis=1)
train = train.drop(['Id'], axis=1) #we don't need ID for our analysis


In [ ]:
missing_train = train.isna().sum().sort_values(ascending=False)
missing_train_percent = ((train.isna().sum() / len(train)) * 100).sort_values(ascending=False)

missing_info = pd.concat([missing_train, missing_train_percent], axis=1, keys=['Total_missing', 'Percent'])
missing_info.head(35)

In [ ]:
train['FireplaceQu'] = train['FireplaceQu'].fillna('None')

garage_features = [feature for feature in train.columns if 'Garage' in feature]
print(garage_features)
for feature in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    train[feature] = train[feature].fillna('None')

for feature in ('GarageCars', 'GarageArea'):
    train[feature] = train[feature].fillna(0)


# train = train.replace({"BsmtExposure" : {"NA" : "None"}})
# train['BsmtExposure'].fillna('None')
bsmt_features = [feature for feature in train.columns if 'Bsmt' in feature]
print(bsmt_features)
for feature in ('BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1', 'BsmtFinType2'):
    train[feature] = train[feature].fillna('None')

for feature in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    train[feature] = train[feature].fillna(0)

#    MasVnrArea and MasVnrType most likely imply no cladding. For numeric - 0, categorical - None
train['MasVnrType'] = train['MasVnrType'].fillna('None')
train['MasVnrArea'] = train['MasVnrArea'].fillna(0)

# Utilities: all entries for Utilities are "AllPub" except for one "NoSeWa", so we deleted this column earlier
# Street: all entries for Street are "Pave" except for 6 "Grvl", so we deleted this column earlier

train['Alley'] = train['Alley'].fillna('None') # We can either do this, or delete the columns with >= 80% null values
train['PoolQC'] = train['PoolQC'].fillna('None')
train['Fence']= train['Fence'].fillna('None')
train['MiscFeature'] = train['MiscFeature'].fillna('None')

#Electrical: only 1 NA value. Since this feature mostly has "SBrkr", we can set it to the missing value.
Electrical_mode = train['Electrical'].mode()[0]
train['Electrical'] = train['Electrical'].fillna(Electrical_mode)

#LotFrontage can be replaced by the median LotFrontage with the neighboorhood, since the area is likely to be similar to other houses in the same area
train['LotFrontage'] = train.groupby('Neighborhood', as_index=False)['LotFrontage'].transform(lambda x: x.fillna(x.median()))
train['GarageYrBlt'] = train.groupby('Neighborhood', as_index=False)['GarageYrBlt'].transform(lambda x: x.fillna(x.median()))

# Some numerical features are actually really categories
train = train.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

train['AgeHouse']=train['YrSold'] - train['YearBuilt']
train['AgeRemodel']=train['YrSold'] - train['YearRemodAdd']
train = pd.get_dummies(train, columns = ['MSSubClass', 'MSZoning', 'Alley', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold',
       'SaleType', 'SaleCondition'])

# Here extreme outliers are being removed
Q1Over=train['OverallQual'].quantile(q=0.25)
Q3Over=train['OverallQual'].quantile(q=0.75)
IQROver=Q3Over-Q1Over

Q1Gr=train['GrLivArea'].quantile(q=0.25)
Q3Gr=train['GrLivArea'].quantile(q=0.75)
IQRGr=Q3Gr-Q1Gr

Q1Cars=train['GarageCars'].quantile(q=0.25)
Q3Cars=train['GarageCars'].quantile(q=0.75)
IQRCars=Q3Cars-Q1Cars

Q1Area=train['GarageArea'].quantile(q=0.25)
Q3Area=train['GarageArea'].quantile(q=0.75)
IQRArea=Q3Area-Q1Area

Q1BsmtSF=train['TotalBsmtSF'].quantile(q=0.25)
Q3BsmtSF=train['TotalBsmtSF'].quantile(q=0.75)
IQRBsmtSF=Q3BsmtSF-Q1BsmtSF

Q1Flr=train['1stFlrSF'].quantile(q=0.25)
Q3Flr=train['1stFlrSF'].quantile(q=0.75)
IQRFlr=Q3Flr-Q1Flr

Lower_Limit_Over = Q1Over - 3*IQROver
Upper_Limit_Over = Q3Over + 3*IQROver

Lower_Limit_Gr = Q1Gr - 3*IQRGr
Upper_Limit_Gr= Q3Gr + 3*IQRGr

Lower_Limit_Cars = Q1Cars - 3*IQRCars
Upper_Limit_Cars = Q3Cars + 3*IQRCars

Lower_Limit_Area = Q1Area - 3*IQRArea
Upper_Limit_Area= Q3Area + 3*IQRArea

Lower_Limit_Bsmt = Q1BsmtSF - 3*IQRBsmtSF
Upper_Limit_Bsmt= Q3BsmtSF + 3*IQRBsmtSF

Lower_Limit_Flr = Q1Flr - 3*IQRFlr
Upper_Limit_Flr= Q3Flr + 3*IQRFlr

train = train[~((train['OverallQual'] < Lower_Limit_Over) | (train['OverallQual'] > Upper_Limit_Over))]
train = train[~((train['GrLivArea'] < Lower_Limit_Gr) | (train['GrLivArea'] > Upper_Limit_Gr))]
train = train[~((train['GarageCars'] < Lower_Limit_Cars) | (train['GarageCars'] > Upper_Limit_Cars))]
train = train[~((train['GarageArea'] < Lower_Limit_Area) | (train['GarageArea'] > Upper_Limit_Area))]
train = train[~((train['TotalBsmtSF'] < Lower_Limit_Bsmt) | (train['TotalBsmtSF'] > Upper_Limit_Bsmt))]
train = train[~((train['1stFlrSF'] < Lower_Limit_Flr) | (train['1stFlrSF'] > Upper_Limit_Flr))]



In [ ]:
missing_train = train.isna().sum().sort_values(ascending=False)
missing_train_percent = ((train.isna().sum() / len(train)) * 100).sort_values(ascending=False)

missing_info = pd.concat([missing_train, missing_train_percent], axis=1, keys=['Total_missing', 'Percent'])
missing_info.head(35)

In [ ]:
print(train['SalePrice'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(train['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
'With this information we can see that the prices are skewed right and some outliers lies above ~500,000.' 

In [ ]:
print("Skewness: %f" % train['SalePrice'].skew())
print("Kurtosis: %f" % train['SalePrice'].kurt())

In [ ]:
'Data is considered to be normal if skewness is between ‐2 to +2 and kurtosis is between ‐7 to +7, hence our data is considered normal.'

In [ ]:
for i in train:
    sns.boxplot(x=i, y="SalePrice", data=train,palette='rainbow')
    plt.show()

In [ ]:
for i in train:
    plt.scatter(x=i, y='SalePrice', data=train,color='blue')
    plt.title(i)
    plt.show()

In [ ]:
sale_price_scaled = StandardScaler().fit_transform(train['SalePrice'][:, np.newaxis])
low = sale_price_scaled[sale_price_scaled[:, 0].argsort()][:10]
high = sale_price_scaled[sale_price_scaled[:, 0].argsort()][-10:]
print(low,'\n\n', high)

In [ ]:
mpl.rcParams['figure.figsize'] = (25,25)
corr_dummy = train.corr()
corr_dummy = np.abs(corr_dummy)
sns.heatmap(corr_dummy)

In [ ]:
corr=train.corr()["SalePrice"]
corr[np.argsort(-1*corr, axis=0)[::-1]].head(80)

In [ ]:
corr_highest=train.corr().unstack().abs().sort_values(ascending=False).drop_duplicates()
corr_highest.head(80)

In [ ]:
'Here we are reducing confunding (wherein variables influence both the dependent variable and independent variable),and only variables uncorrelated with each other should be added to regression models (which are correlated with SalePrice). I have eliminated features that have a confounders with a correlation of 0.8 or higher among other independent variables. I have eliminated GarageArea, 1stFlrSF, and TotRmsAbvGrd.'

In [ ]:
'OverallQual, GrLivArea, GarageCars, TotalBsmtSF, FullBath, BsmtQual_Ex, AgeHouse, KitchenQual_TA, AgeRemodel, KitchenQual_Ex, MasVnrArea, Fireplaces, ExterQual_Gd, ExterQual_Ex, HeatingQC_Ex, Neighborhood_NridgHt, BsmtFinSF1, MSSubClass_SC60, MasVnrType_None, anc SaleType_New have more than an absolute value correlation of 0.35 or above with SalePrice.'

In [ ]:
x = train[['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','FullBath','BsmtQual_Ex','AgeHouse','KitchenQual_TA','AgeRemodel','KitchenQual_Ex','MasVnrArea','Fireplaces','ExterQual_Gd','ExterQual_Ex','HeatingQC_Ex','Neighborhood_NridgHt','BsmtFinSF1','MSSubClass_SC60','MasVnrType_None','SaleType_New']]
y = train['SalePrice']
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()
print(model.summary())

train['Prediction']=model.predict()

In [ ]:
# train = pd.read_csv('train.csv')

train['Prediction']=model.predict()

ax1=train[['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','FullBath','BsmtQual_Ex']].plot()
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
ax2=ax1.twinx()
ax2.plot(train['Prediction'],color='red',label='Prediction')
ax2.plot(train['SalePrice'],color='black',label='Sale Price')
ax2.xaxis.set_major_locator(plt.MaxNLocator(10))
ax1.legend()
ax2.legend(bbox_to_anchor=(1.019,1), loc="upper left")
plt.title('Regressive Equation Plot')
plt.show()



In [ ]:
train.to_csv("train_clean15.csv")